In [1]:
import pandas as pd
import os
import wiggum as wg
import numpy as np

In [2]:
labeled_df = wg.LabeledDataFrame('../data/iristest')

In [3]:
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
sepal length,float64,continuous,"[independent, dependent]",False,NaN
sepal width,float64,continuous,[dependent],False,NaN
petal length,float64,continuous,[independent],False,NaN
petal width,float64,continuous,[independent],False,NaN
class,object,categorical,[splitby],False,NaN


In [4]:
corrobj = wg.All_Pearson()
corrobj.get_trend_vars(labeled_df)
corrobj.regression_vars

[('sepal length', 'sepal length'),
 ('sepal length', 'sepal width'),
 ('petal length', 'sepal length'),
 ('petal length', 'sepal width'),
 ('petal width', 'sepal length'),
 ('petal width', 'sepal width')]

In [5]:
labeled_df.get_subgroup_trends_1lev([corrobj])
labeled_df.result_df.head()

,dependent,group_feat,independent,subgroup,subgroup_trend,subgroup_trend_strength,trend_type,agg_trend,agg_trend_strength,comparison_type
0,sepal length,class,sepal length,Iris-setosa,1.000000,1.000000,pearson_corr,1.000000,1.000000,aggregate-subgroup
1,sepal width,class,sepal length,Iris-versicolor,0.525911,0.525911,pearson_corr,-0.109369,0.109369,aggregate-subgroup
2,sepal length,class,petal length,Iris-virginica,0.864225,0.864225,pearson_corr,0.871754,0.871754,aggregate-subgroup


In [6]:
indep, dep = zip(*corrobj.regression_vars)
corr_var_list = list(set(indep))
corr_var_list.extend(list(set(dep)))
corr_var_list = list(set(corr_var_list))

In [7]:
corr_var_list

['sepal length', 'sepal width', 'petal width', 'petal length']

In [8]:
corrmat = labeled_df.df[corr_var_list].corr()
corrmat

,sepal length,sepal width,petal width,petal length
sepal length,1.000000,-0.109369,0.817954,0.871754
sepal width,-0.109369,1.000000,-0.356544,-0.420516
petal width,0.817954,-0.356544,1.000000,0.962757
petal length,0.871754,-0.420516,0.962757,1.000000


In [9]:
corrmat['sepal width']['sepal width']

1.0

In [12]:
for i,d in corrobj.regression_vars:
    print(i,d, ':',corrmat[i][d])

sepal length sepal length : 1.0
sepal length sepal width : -0.10936924995064935
petal length sepal length : 0.8717541573048719
petal length sepal width : -0.42051609640115484
petal width sepal length : 0.8179536333691635
petal width sepal width : -0.3565440896138055
